In [3]:
# go wide screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [234]:
import time
def pp(s):
    print(s)
    return s + "\n"


In [4]:
import numpy as np

In [5]:
import spectrum

In [6]:
from matplotlib import pyplot as plt
% matplotlib inline

In [7]:
import pyregion
from astropy.io import fits

In [8]:
fregion = "LAB.reg"

with open(fregion) as fin:
    region_string = fin.read()

In [9]:
fcube = "data/s2outcube_median_022.fits.gz"
s = spectrum.readSpectrum(fcube)
ww = s.grid()

In [ ]:
r2 = pyregion.parse(region_string).as_imagecoord(s.hdu.header)
myfilter = r2.get_filter()

mask = r2.get_mask(shape=c[0].shape)



In [ ]:

plt.subplot(121).imshow(mask, origin="lower", interpolation="nearest")


In [ ]:
sout = np.zeros( c.shape[0]  )
for i in range(c.shape[0]):
    sout[i] = np.sum( c[i][mask] )

In [ ]:
def draw_line(wl, label):
    plt.text(wl,100.,label, rotation=90., ha='center', size=20, va='top')
    plt.plot([wl]*2,[0.,80.], c='r', lw=1.)

In [ ]:
f = plt.figure(figsize = [15,10])
plt.plot(ww, sout, drawstyle='steps-mid')
plt.ylim([-30,100])
plt.xlabel("wl[A]")
plt.ylabel("flux [arb.]")

#If line 3844.0A is [Lyα         ] at rest. wl = 1215.2A (z = 2.163)
# Line 4906.7 could be [C IV        ] at rest wl. 1549.48, obs. wl = 4901.25A, dA = 5.45A
# Line 5193.4 could be [He II       ] at rest wl. 1640.4, obs. wl = 5188.85A, dA = 4.55A


draw_line(3844.0, "Lyα")
draw_line(4901.25, "[C IV]")
draw_line(5188.85, "[He II]")
plt.xlim([3500.,5250.])

#plt.xlim([-30,100])

In [ ]:
f = plt.figure(figsize = [15,10])
plt.plot(ww, sout, drawstyle='steps-mid')
plt.ylim([-30,100])
plt.xlabel("wl[A]")
plt.ylabel("flux [arb.]")

#If line 3844.0A is [Lyα         ] at rest. wl = 1215.2A (z = 2.163)
# Line 4906.7 could be [C IV        ] at rest wl. 1549.48, obs. wl = 4901.25A, dA = 5.45A
# Line 5193.4 could be [He II       ] at rest wl. 1640.4, obs. wl = 5188.85A, dA = 4.55A


draw_line(3844.0, "Lyα")
draw_line(4901.25, "[C IV]")
draw_line(5188.85, "[He II]")
plt.xlim([3500.,5250.])


In [ ]:
from collections import OrderedDict

# Here the actual cloudfinder happens

In [ ]:
# old object oriented routine, slow
class Pixel():
    def __init__(self, c, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.c = c
        self.explored = False
        
    def __str__( self ):
        return "({},{},{})".format(self.x, self.y, self.z)
    
    def get_coor_tuple(self):
        return (self.x, self.y, self.z) 
        
    def get_children(self):
        children = []
        ddx = [-1,0,1]
        ddy = [-1,0,1]
        ddz = [-1,0,1]
        for dx in ddx:
            for dy in ddy:
                for dz in ddz:
                    if not dx == dy == dz == 0:
                        newx, newy, newz = self.x+dx, self.y+dy, self.z+dz
                        if newx >= 0 and newx < c.shape[2]\
                            and newy >= 0 and newy < c.shape[1]\
                            and newz >= 0 and newz < c.shape[0]:
                               children.append( Pixel(self.c, newx, newy, newz) ) 
        return children
    
    def get_cube_val(self):
        return c[self.z, self.y, self.x]
    
    def build_map(self, threshold, label):
        outmap = np.zeros_like(self.c, dtype=int)
        visited = np.array( np.zeros_like(self.c), dtype=bool)
        pixel_list = []
        stack = [self]
        #visited = [self.get_coor_tuple()]
        
        maxiter = 1e9
        iter = 0
        while len(stack) > 0: 
            #print("{} elements on stack, visited {} pixels.".format(len(stack), len(visited)))
            #stack[(self.x,self.y,self.z)] = self
            pm = stack.pop()
            #visited.append( pm.get_coor_tuple() )
            visited[pm.z, pm.y, pm.x] = True
            #print("Pixel {}".format(self))
            pp = pm.get_children()
            #print("{} children".format(len(pp)))
            #N =  sum( [p.get_cube_val() > threshold for p in pp] ) 
            #print("{} are larger than threshold.".format(N))
            for p in pp:
                #print(p, p.get_cube_val() )
                if p.get_cube_val() > threshold and not visited[p.z, p.y, p.x]:
                    #print("  val: ", p.get_cube_val())
                    
                    stack.append(p)
                    outmap[p.z, p.y, p.x] = label
                    pixel_list.append( [p.x, p.y, p.z, p.get_cube_val()] )
            #print("{} elements on stack, visited {} pixels.".format(len(stack), len(visited)))
            iter += 1
            if iter > maxiter:
                break
            
        return outmap, pixel_list
            
    def build_map(self, threshold, label):
        outmap = np.zeros_like(self.c, dtype=int)
        visited = np.array( np.zeros_like(self.c), dtype=bool)
        pixel_list = []
        stack = [self]
        #visited = [self.get_coor_tuple()]
        
        maxiter = 1e9
        iter = 0
        while len(stack) > 0: 
            #print("{} elements on stack, visited {} pixels.".format(len(stack), len(visited)))
            #stack[(self.x,self.y,self.z)] = self
            pm = stack.pop()
            #visited.append( pm.get_coor_tuple() )
            visited[pm.z, pm.y, pm.x] = True
            #print("Pixel {}".format(self))
            pp = pm.get_children()
            #print("{} children".format(len(pp)))
            #N =  sum( [p.get_cube_val() > threshold for p in pp] ) 
            #print("{} are larger than threshold.".format(N))
            for p in pp:
                #print(p, p.get_cube_val() )
                if p.get_cube_val() > threshold and not visited[p.z, p.y, p.x]:
                    #print("  val: ", p.get_cube_val())
                    
                    stack.append(p)
                    outmap[p.z, p.y, p.x] = label
                    pixel_list.append( [p.x, p.y, p.z, p.get_cube_val()] )
            #print("{} elements on stack, visited {} pixels.".format(len(stack), len(visited)))
            iter += 1
            if iter > maxiter:
                break
            
        return outmap, pixel_list            

In [220]:
def build_map2(c, outmap, x, y, z, threshold, label):
    # Non object oriented version
    def get_children(zyx, shape):
        maxz, maxy, maxx = shape
        z, y, x = zyx
        children = []
        ddx = [-1,0,1]
        ddy = [-1,0,1]
        ddz = [-1,0,1]
        #ddz = [0]
        for dx in ddx:
            for dy in ddy:
                for dz in ddz:
                    if not dx == dy == dz == 0:
                        newx, newy, newz = x+dx, y+dy, z+dz
                        if newx >= 0 and newx < maxx\
                            and newy >= 0 and newy < maxy\
                            and newz >= 0 and newz < maxz:
                               children.append( (newz, newy, newx) ) 
        return children

    pixel_list = []
    stack = [(z,y,x)]
    outmap[(z,y,x)] = label

    maxiter = 1e6
    iter = 0
    while len(stack) > 0: 
        pm = stack.pop()
        pp = get_children(pm, c.shape)
        for p in pp:
            if c[p] >= threshold and  outmap[p] == 0: # pixel that are labeled 0 have not been visited yet
                stack.append(p)
                outmap[p] = label
            elif c[p] < threshold and  outmap[p] == 0:
                outmap[p] = -1 # label pixel with -1 if they have beed visited already
        iter += 1
        if iter > maxiter:
            break

    return outmap#pixel_list            

In [235]:
# test HD-LAB1
z0 = 176
x0 = 91
y0 = 49
  
c = s.data
    

grow_threshold = 0.15

summary = ""
summary += pp("Building map ...")
start_time = time.time()
#m,pl = p0.build_map(threshold = 0.2, label=1)

outmap = np.zeros_like(c, dtype=int)

m = build_map2(c, outmap, x0, y0, z0, threshold = grow_threshold, label=1)

time_to_build = time.time() - start_time
summary += pp("Time to build map: {:.4e} s".format(time_to_build))

print("{} labeled pixels on map".format( np.nansum( (m>0).flatten())) )
print("{} untouched pixels on map".format( np.nansum( (m == 0).flatten())) )
print("{} visited but unlabeled pixels on map".format( np.nansum( (m == -1).flatten())) )

Building map ...
Time to build map: 2.0494e-01 s
4170 labeled pixels on map
21944643 untouched pixels on map
4547 visited but unlabeled pixels on map


In [74]:
# test
z0 = 342
x0 = 80
y0 = 113

c = s.data
    
grow_threshold = 0.5

summary = ""
summary += pp("Building map ...")
start_time = time.time()

m = build_map2(c, outmap, x0, y0, z0, threshold = grow_threshold, label=2)

time_to_build = time.time() - start_time
summary += pp("Time to build map: {:.4e} s".format(time_to_build))

print("{} labeled pixels on map".format( np.nansum( (m>0).flatten())) )
print("{} untouched pixels on map".format( np.nansum( (m == 0).flatten())) )
print("{} visited but unlabeled pixels on map".format( np.nansum( (m == -1).flatten())) )

Building map ...
Time to build map: 1.1640e-03 s
4212 labeled pixels on map
21944461 untouched pixels on map
4687 visited but unlabeled pixels on map


In [68]:
#test
f = np.zeros_like(c)
f[m > 0] = c[m > 0]

# shell
#f[m == -1] = c[m == -1]

h = fits.PrimaryHDU(data=f, header=s.hdu.header)
h.header["CD1_1"] = -0.00013888888888888   
h.header["CD1_2"] = 0. 
h.header["CD2_2"] = 0.000138888888888888  
h.header["CD2_1"] = 0.
h.header["CD3_3"] = 1.985839800000122  
h.writeto("m.fits.gz", overwrite=True)

In [188]:
c = s.data

c[898:923] = 0. # take out bad region

In [249]:
detect_threshold = 0.5
#grow_threshold = 0.274
grow_threshold = 0.3

detect_threshold = 0.6
#grow_threshold = 0.274
grow_threshold = 0.4

In [250]:
# run cloud growth algorithm on all pixels above detection threshold
# try to be intelligent, only loop over pixels that exceed detection threshold
ii = c > detect_threshold 
N =  (np.sum(ii))
print("{} pixel above detection threshold".format(N))

outmap = np.zeros_like(c, dtype=int)

zz,yy,xx = [np.arange(s, dtype=int) for s in c.shape]

YY,ZZ,XX = np.meshgrid(yy, zz, xx)

label = 1
for i, (x,y,z) in enumerate( zip(XX[ii], YY[ii], ZZ[ii] ) )  :  
    if outmap[z,y,x] == 0:
        pp("### {} ###".format(i))
        print(x,y,z, outmap[z,y,x])
        #outmap = build_map2(c, outmap, x,y,z, threshold = grow_threshold, label=label)
        
        summary = ""
        summary += pp("Building map starting with pixel {} out of {} that exceeds threshold...".format(i,N))
        start_time = time.time()
        #print("{} labeled pixels on map".format( np.nansum( (outmap > 0).flatten())) )
        outmap = build_map2(c, outmap, x,y,z, threshold = grow_threshold, label=label)
        time_to_build = time.time() - start_time
        summary += pp("Time to build map: {:.4e} s".format(time_to_build))

        print("{} labeled pixels on map".format( np.nansum( (outmap > 0).flatten())) )
        print("{} untouched pixels on map".format( np.nansum( (outmap == 0).flatten())) )

    label += 1
    
    if i > 1e6:
        break
#print (cnt)

77346 pixel above detection threshold
### 0 ###
97 119 29 0
Building map starting with pixel 0 out of 77346 that exceeds threshold...
Time to build map: 3.3797e-02 s
265 labeled pixels on map
21952470 untouched pixels on map
### 3 ###
98 118 77 0
Building map starting with pixel 3 out of 77346 that exceeds threshold...
Time to build map: 8.2867e+00 s
60405 labeled pixels on map
21852944 untouched pixels on map
### 1598 ###
90 47 173 0
Building map starting with pixel 1598 out of 77346 that exceeds threshold...
Time to build map: 7.2379e-02 s
60972 labeled pixels on map
21851701 untouched pixels on map
### 2475 ###
23 62 197 0
Building map starting with pixel 2475 out of 77346 that exceeds threshold...
Time to build map: 6.9512e-02 s
61485 labeled pixels on map
21850271 untouched pixels on map
### 3969 ###
60 140 242 0
Building map starting with pixel 3969 out of 77346 that exceeds threshold...
Time to build map: 2.4098e-02 s
61685 labeled pixels on map
21849681 untouched pixels on map


In [251]:
# filter regions with too small volume 
minsize = 3
rr = np.sort( np.unique( outmap.flatten() ) )
for r in rr:
    if not r > 0:
        continue
    N = np.sum( outmap == r )
    #print("{} : N = {}".format(r, N))
    if N < minsize:
        outmap[outmap == r] = -1  
rr = np.sort( np.unique( outmap.flatten() ) )

# relabel
for i,r in enumerate(rr[rr>0]):
    outmap[outmap == r] = i + 1
    
rr = np.sort( np.unique( outmap.flatten() ) )
print("{} regions survive size cut".format( len(rr[rr>0]) ))


29 regions survive size cut


In [252]:
w.wcs_pix2world([[100,100,178]],1)

array([[  1.50185241e+02,   2.14922796e+00,   3.84623364e+03]])

In [260]:
# build catalog

from astropy.table import Table

names   = [ "id", "N", "flux", "ra_com", "dec_com", "ddec", "dra", "x_com", "y_com", "z_com", "dx", "dy", "dz", "wl_com", "dwl", "xmin","xmax", "ymin", "ymax", "zmin", "zmax"]
t = Table(names=names)

dtypes   =  {"id" : int, "N" : int, "flux" : float, 
             "ra_com" : float, "dec_com" : float, 
             "dra" : float, "ddec" : float,
             "x_com": int, "y_com": int, "z_com": int, 
             "dx": int, "dy": int, "dz": int, "wl_com" : float, 
             "dwl": float, 
             "xmin": int,"xmax": int, "ymin": int, "ymax": int, "zmin": int, "zmax": int}
formats  =  {"id" : "5d", "N" : "5d", "flux" : ".4e", "ra_com" : "12.6f", "dec_com" : "12.6f", "dra" : "3.1f", "ddec" : "3.1f", "x_com": "8.2f", "y_com": "8.2f", "z_com": "8.2f", "dx": "4d", "dy": "4d", "dz": "4d", "wl_com" : "8.2f", "dwl": "8.2f", "xmin": "4d","xmax": "4d", "ymin": "4d", "ymax": "4d", "zmin": "4d", "zmax": "4d"}
units    =  {"id" : "", "N" : "px", "flux" : "arb", "ra_com" : "Deg[J2000]", "dec_com" : "Deg[J2000]", "dra" : "arcsec", "ddec" : "arcsec", "x_com": "px", "y_com": "px", "z_com": "px", "dx": "px", "dy": "px", "dz": "px", "wl_com" : "A", "dwl": "A", "xmin": "px","xmax": "px", "ymin": "px", "ymax": "px", "zmin": "px", "zmax": "px"}


from astropy import wcs
w = wcs.WCS(s.hdu)
platescale = s.hdu.header["CDELT2"]

for n in names:
    t[n].dtype  = dtypes[n]
    t[n].format = formats[n]
    t[n].unit =  units[n]

# Generate catalog
for r in rr[rr>0]:
    ii = outmap == r
    N = np.sum(ii)
    M = np.sum(c[ii])
    x_com = np.sum(c[ii]*XX[ii])/M
    y_com = np.sum(c[ii]*YY[ii])/M
    z_com = np.sum(c[ii]*ZZ[ii])/M
    
    wl_com = float(ww_interp(z_com))
    zmax,zmin = int(ZZ[ii].max()) , int(ZZ[ii].min())
    ymax,ymin = int(YY[ii].max()) , int(YY[ii].min())
    xmax,xmin = int(XX[ii].max()) , int(XX[ii].min())
    dz = zmax - zmin
    dx = xmax - xmin
    dy = ymax - ymin
    
    # use atropy wcs to convert x,y,z, dz to ra,dec,wl  ... and in bit of a hack ... dwl
    voxel = w.wcs_pix2world([[x_com,y_com,z_com],[x_com,y_com,ZZ[ii].max()],[x_com,y_com,ZZ[ii].min()]],1)
    dwl = voxel[1][2] - voxel[2][2]
    ra_com, dec_com,wl_com = voxel[0]
    
    dra = platescale*dx * 3600.
    ddec = platescale*dy * 3600.
    t.add_row([ r, N, M, ra_com, dec_com, dra, ddec, x_com, y_com, z_com, dx, dy, dz, wl_com, dwl, xmin,xmax, ymin, ymax, zmin, zmax] )
    
t

id,N,flux,ra_com,dec_com,ddec,dra,x_com,y_com,z_com,dx,dy,dz,wl_com,dwl,xmin,xmax,ymin,ymax,zmin,zmax
,px,arb,Deg[J2000],Deg[J2000],arcsec,arcsec,px,px,px,px,px,px,A,A,px,px,px,px,px,px
int64,int64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,int64,int64,int64,int64,int64,int64
1,265,1.1943e+02,150.185545,2.151851,2.0,2.0,97.00,118.00,37.00,4,4,26,3566.70,51.63,96,100,117,121,26,52
2,60140,5.5759e+04,150.185589,2.151713,7.0,9.5,97.00,117.00,547.00,14,19,831,4580.54,1650.23,89,103,105,124,66,897
3,567,3.0554e+02,150.186446,2.142081,6.0,5.0,91.00,48.00,175.00,12,10,8,3842.16,15.89,85,97,43,53,172,180
4,513,2.3867e+02,150.196041,2.143887,3.5,3.0,22.00,61.00,197.00,7,6,33,3885.33,65.53,19,26,58,64,179,212
5,200,9.4023e+01,150.190697,2.154851,3.5,2.5,60.00,140.00,243.00,7,5,9,3975.33,17.87,57,64,138,143,239,248
6,42914,2.9885e+04,150.190700,2.154846,9.0,7.5,60.00,140.00,643.00,18,15,630,4770.78,1251.08,52,70,133,148,267,897
7,570,2.8478e+02,150.196007,2.144031,5.0,6.0,22.00,62.00,287.00,10,12,15,4062.93,29.79,16,26,59,71,279,294
8,636,3.1407e+02,150.196053,2.144134,3.5,4.5,22.00,63.00,311.00,7,9,23,4111.44,45.67,19,26,59,68,300,323


In [261]:
# save map
f = np.zeros_like(c)
f[outmap > 0] = c[outmap > 0]

# shell
#f[m == -1] = c[m == -1]

h = fits.PrimaryHDU(data=outmap, header=s.hdu.header)
h.header["CD1_1"] = -0.00013888888888888   
h.header["CD1_2"] = 0. 
h.header["CD2_2"] = 0.000138888888888888  
h.header["CD2_1"] = 0.
h.header["CD3_3"] = 1.985839800000122  
h.writeto("outmap.fits.gz", overwrite=True)

In [262]:
# save map filtered data
f = np.zeros_like(c)
f[outmap > 0] = c[outmap > 0]

# shell
#f[m == -1] = c[m == -1]

h = fits.PrimaryHDU(data=f, header=s.hdu.header)
h.header["CD1_1"] = -0.00013888888888888   
h.header["CD1_2"] = 0. 
h.header["CD2_2"] = 0.000138888888888888  
h.header["CD2_1"] = 0.
h.header["CD3_3"] = 1.985839800000122  
h.writeto("mapfiltered.fits.gz", overwrite=True)

In [263]:
# save shells 
# shell
f = np.zeros_like(c)
f[outmap == -1] = c[outmap == -1]

h = fits.PrimaryHDU(data=f, header=s.hdu.header)
h.header["CD1_1"] = -0.00013888888888888   
h.header["CD1_2"] = 0. 
h.header["CD2_2"] = 0.000138888888888888  
h.header["CD2_1"] = 0.
h.header["CD3_3"] = 1.985839800000122  
h.writeto("shells.fits.gz", overwrite=True)

In [ ]:
sum(visited.flatten())

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
% matplotlib notebook
fig = plt.figure(figsize=[15,10])
ax = fig.add_subplot(111, projection='3d')
ax.scatter3D(pl[:,0], pl[:,1], pl[:,2],c=pl[:,3], alpha=.1)
plt.show()